# Introdução

## Importação das bibliotecas necessarias

In [42]:
%matplotlib inline
import requests
import pandas as pd
import os
import json
import sys
import matplotlib.pyplot as plt

## Funções auxiliares

In [43]:
def dot_print():
    sys.stdout.write('.')
    sys.stdout.flush()

# Dataframe base de Código do município e Ano

In [44]:
ano_df = pd.DataFrame([2012,2013,2014,2015,2016,2017,2018,2019,2020], columns= ['ANO'])
fetch_mun_df = pd.read_excel("../../csv_files/tb_municipios.xlsx")
mun_df = pd.DataFrame([], columns = ['CODIGO_MUNICIPIO'])
mun_df['CODIGO_MUNICIPIO'] = fetch_mun_df['codigo_municipio_completo']
base_df = pd.merge(mun_df.assign(key=1), ano_df.assign(key=1),on='key').drop('key', axis=1)
base_df['CODIGO_MUNICIPIO'] = base_df['CODIGO_MUNICIPIO'].astype(str)
base_df['CODMUNRES'] = base_df['CODIGO_MUNICIPIO'].replace(r'.$', '', regex=True)
base_df = base_df.sort_values(by=['CODMUNRES', 'ANO'])
base_df.head()

,CODIGO_MUNICIPIO,ANO,CODMUNRES
0,1100015,2012,110001
1,1100015,2013,110001
2,1100015,2014,110001
3,1100015,2015,110001
4,1100015,2016,110001


# Tratamento das planilhas de deslocamento para o parto

In [57]:
df_mun_array = []
df_uf_array = []
for i in range(9):
    dot_print()
    df_mun_aux = pd.read_excel("./BR"+str(12+i)+"dc_Rosa.ods")
    df_mun_aux.insert(1, 'ANO', [2012+i] * len(df_mun_aux.index), True)
    df_mun_array.append(df_mun_aux)
    dot_print()
    df_uf_aux = pd.read_excel("./Brasil-UFs-12a20_Rosa.ods", sheet_name="BR"+str(12+i))
    df_uf_aux.insert(1, 'ANO', [2012+i] * len(df_uf_aux.index), True)
    df_uf_array.append(df_uf_aux)
print('(+)')

df_mun = pd.concat(df_mun_array, ignore_index=True)
df_uf = pd.concat(df_uf_array, ignore_index=True)

..................(+)


In [58]:
df_uf2 = df_uf.dropna(subset=['NOME'])
df_uf2 = df_uf2.fillna({'CODUFr': 55})
df_uf2 = df_uf2.fillna(0)
df_uf2 = df_uf2.sort_values(by=['CODUFr', 'ANO'])
df_uf2 = df_uf2.rename(columns={"CODUFr": "UF"})
#df_uf2.head()
df_uf2.to_csv('./indicadores_bloco4_deslocamento_parto_UF_2012-2020.csv',index=False)

In [59]:
df_mun2 = df_mun.sort_values(by=['CODMUNRES', 'ANO'])
df_mun2 = df_mun2.iloc[:-7, :]
del df_mun2['NOME']
base_df['CODMUNRES']=base_df['CODMUNRES'].astype(int)
df_mun2['CODMUNRES']=df_mun2['CODMUNRES'].astype(int)
df_mun3 = pd.merge(base_df, df_mun2, how='left', on=['CODMUNRES','ANO'])
del df_mun3['CODIGO_MUNICIPIO']
df_mun3 = df_mun3.sort_values(by=['CODMUNRES', 'ANO'])
#df_mun3.head()
df_mun3.to_csv('./indicadores_bloco4_deslocamento_parto_municipio_2012-2020.csv',index=False)

In [60]:
df_void = pd.read_excel('./Incompletitude_indicadores_deslocamento_parto.xlsx', sheet_name='BD')

In [70]:
df_void1 = df_void.rename(columns={"ano":"ANO", "uf": "UF", "mun_res": "CODMUNRES"})
df_void1['CODMUNRES'] = df_void1['CODMUNRES'].astype(int)
df_void2 = pd.merge(base_df, df_void1, how='left', on=['CODMUNRES', 'ANO'])
df_void2 = df_void2.sort_values(by=['CODMUNRES', 'ANO'])
df_void2 = df_void2.fillna(0)
del df_void2['CODIGO_MUNICIPIO']

In [71]:
df_void2.to_csv('./Incompletitude_indicadores_deslocamento_parto.csv', index=False)